In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from pymongo import MongoClient
import configparser
import logging
import logging.config
sys.path.append(os.path.join(os.path.dirname('__file__'),'..'))
os.environ['INIDIR'] = '/Users/macico/Dropbox/kaggle/kuloko/ini'
os.environ['LOGDIR'] = '/Users/macico/Dropbox/kaggle/kuloko/log'
INIDIR=os.environ['INIDIR'] 
LOGDIR=os.environ['LOGDIR'] 

logging.config.fileConfig(os.path.join(INIDIR,'logconfig.ini'),defaults={'logfilename': os.path.join(LOGDIR,'logging.log')})
logger = logging.getLogger("KULOKO")


In [3]:
mongo_ini = configparser.ConfigParser()
mongo_ini.read(os.path.join(INIDIR,'db_config.ini'), encoding='utf-8')

from src.mongo_handler import MongoHandler

In [4]:

import  kuloko_handler.handler.socket_handler as skt_api

private_api_ini = configparser.ConfigParser()
private_api_ini.read(os.path.join(INIDIR,'private_api.ini'), encoding='utf-8')

general_config_ini = configparser.ConfigParser()
general_config_ini.read(os.path.join(INIDIR,'config.ini'), encoding='utf-8')


['/Users/macico/Dropbox/kaggle/kuloko/ini/config.ini']

In [1]:
# mongo_db = MongoHandler(mongo_ini['LOCAL'],"trade")
# mongo_db.insert_one({"test":"rrr"})
# for db_data in mongo_db.find():
#     print(db_data)
# mongo_db.delete_all()

In [6]:
trades = skt_api.Trade(logger, general_config_ini,private_api_ini)
trades.connect(trades.get_url(),'BTC')
trades.subscribe()
mongo_db = MongoHandler(mongo_ini['LOCAL'],"trade")

In [11]:
import time
retry_cnt=0
max_retry_cnt=120
print("START")
while True:
    try:
        time.sleep(10)
        data = trades.get()
        print("Data:{0}, Retry_cnt={1}".format(len(data),retry_cnt))
        if len(data)==0:
            retry_cnt += 1
            continue
        insert_json =[ trades.convert_shape(_data,"json") for _data in data ]
        mongo_db.insert_many(insert_json)
        retry_cnt = 0
    except Exception as e:
        retry_cnt += 1
        print("ERROR",e)
        if retry_cnt < max_retry_cnt:
            continue
        print("END",e)
        break


START
HTTP/1.1 101 Switching Protocols
Date: Sun, 06 Dec 2020 11:37:46 GMT
Connection: upgrade
Upgrade: websocket
Sec-WebSocket-Accept: YUDTgrBcS0sTF/Tfly1HIekVNsE=
X-Cache: Miss from cloudfront
Via: 1.1 d8e9b6af48fa5d844b0f68fc85959592.cloudfront.net (CloudFront)
X-Amz-Cf-Pop: NRT12-C5
X-Amz-Cf-Id: ZLs2FMGEvUeNxGjB-pKcZuxsoS-fvPijnH6RH04VLmGLeKMwU7dUpA==
-----------------------
send: b'\x81\xbeb\xf6\xc6\xa5\x19\xd4\xa5\xca\x0f\x9b\xa7\xcb\x06\xd4\xfc\x85@\x85\xb3\xc7\x11\x95\xb4\xcc\x00\x93\xe4\x89B\xd4\xa5\xcd\x03\x98\xa8\xc0\x0e\xd4\xfc\x85@\x82\xb4\xc4\x06\x93\xb5\x87N\xd6\xe4\xd6\x1b\x9b\xa4\xca\x0e\xd4\xfc\x85@\xb4\x92\xe6@\x8b'
Data:0, Retry_cnt=0
Data:2, Retry_cnt=1
Data:6, Retry_cnt=0
Data:4, Retry_cnt=0
Data:0, Retry_cnt=0
send: b'\x8a\x80\xd0+\xc6x'
Data:2, Retry_cnt=1
send: b'\x8a\x80\xab^;\xe1'
Data:0, Retry_cnt=0
Data:4, Retry_cnt=1
Data:0, Retry_cnt=0
Data:1, Retry_cnt=1
Data:0, Retry_cnt=0
send: b'\x8a\x80\xac\x8f\xb4\xab'
Data:1, Retry_cnt=1
send: b'\x8a\x80H\xfa\x01\x

In [ ]:
trades.disconnect()